<a href="https://colab.research.google.com/github/LastResort/114_bi_homework/blob/main/5113029006_week11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝yahoo finance套件
!pip install yfinance

In [ ]:
import google.generativeai as genai  # Google生成式AI套件Gemini API
import os  # 系統檔，用來操作檔案與文件
import requests  # 用來發HTTP請求，用於網路爬蟲
from bs4 import BeautifulSoup  # 網路爬蟲套件
import numpy as np  # 數值計算
import pandas as pd  # 資料處理
import yfinance as yf  # Yahoo finance 股價資訊
import datetime as dt  # 處理日期時間
import time  # 時間延遲套件，用於控制API呼叫頻率


# 爬取股票資訊的類別
class StockInfo():
    """
    股票基本資訊爬取類別

    功能：
    1. 從台灣證券交易所爬取所有上市公司的股號、股名、產業別
    2. 提供股號查詢股名的功能

    資料來源：台灣證券交易所 (TWSE) ISIN網站
    """

    # 取得全部股票的股號、股名
    def stock_name(self):
        """
        爬取所有台股的股號、股名、產業別

        Returns:
            DataFrame: 包含「股號」、「股名」、「產業別」三欄的資料表

        技術說明：
        - 使用requests取得網頁內容
        - 使用BeautifulSoup解析HTML
        - 過濾出4碼股號的上市公司（排除ETF等其他商品）
        """
        # 向台灣證券交易所發送請求
        response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
        # 使用BeautifulSoup解析HTML內容
        url_data = BeautifulSoup(response.text, 'html.parser')
        # 找出所有的表格列（tr標籤）
        stock_company = url_data.find_all('tr')

        # 資料處理：提取股號、股名、產業別
        # \u3000 是全形空白字元，用來分隔股號和股名
        # 只保留4碼的股號（過濾掉ETF、特別股等）
        data = [
            (row.find_all('td')[0].text.split('\u3000')[0].strip(),
                row.find_all('td')[0].text.split('\u3000')[1],
                row.find_all('td')[4].text.strip())
            for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
        ]
        # 建立DataFrame並設定欄位名稱
        df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])
        return df

    # 取得股票名稱
    def get_stock_name(self, stock_id, name_df):
        """
        根據股號查詢股名

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            name_df (DataFrame): 包含股號和股名的資料表

        Returns:
            str: 股票名稱（例如：'台積電'）
        """
        return name_df.set_index('股號').loc[stock_id, '股名']

In [ ]:
class StockAnalysis():
    """
    股票分析類別 - 整合股價資料與Gemini AI分析

    主要功能：
    1. 取得股票價格資料（技術面）
    2. 取得股票基本面資料（營收、EPS）
    3. 使用Gemini AI進行股票分析
    4. 多檔股票比較與推薦

    設計理念：
    - 整合多元數據源（價格、基本面）
    - 利用LLM進行智能分析
    """

    def __init__(self, gemini_api_key):
        """
        初始化股票分析系統

        Args:
            gemini_api_key (str): Google Gemini API金鑰

        初始化步驟：
        1. 設定Gemini API
        2. 建立生成模型實例
        """
        # 設定Gemini API金鑰
        genai.configure(api_key=gemini_api_key)
        """
        1. Gemini 2.5 Pro - 進階思考模型
        2. Gemini 2.5 Flash - 快速且智慧
        3. Gemini 2.5 Flash-Lite - 速度最快的 Flash 模型，可提高成本效益和輸送量。
        """
        # 使用Gemini 2.5 Pro模型（高階推理能力）
        self.model = genai.GenerativeModel('gemini-2.5-pro')
        # 建立股票資訊物件
        self.stock_info = StockInfo()
        # 預先載入所有股票名稱（避免重複爬取）
        self.name_df = self.stock_info.stock_name()


    def stock_price(self, stock_id, days=15):
        """
        取得股票近期價格資料（技術面分析基礎）

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            days (int): 要取得的天數（預設15天）

        Returns:
            dict: 包含日期、收盤價、每日報酬率的字典

        技術說明：
        - 使用yfinance API取得Yahoo Finance資料
        - 計算每日報酬率 = (今日收盤-昨日收盤)/昨日收盤
        - 自動加上.TW後綴（台股代碼格式）
        """

        # 台股代碼需要加上.TW後綴
        stock_id += '.TW'
        # 計算日期區間
        end = dt.date.today() # 今天
        start = end - dt.timedelta(days=days) # 往前推N天

        # 禁用 FutureWarning
        import warnings
        warnings.filterwarnings('ignore', category=FutureWarning)

        # 下載股價資料
        # auto_adjust=True 會自動調整除權息的價格
        df = yf.download(stock_id, start=start, auto_adjust=True)

        # 將欄位名稱改為中文（方便閱讀）
        df.columns = ['開盤價', '最高價', '最低價', '收盤價', '成交量']
        # 建立回傳的資料結構
        data = {
            '日期': df.index.strftime('%Y-%m-%d').tolist(), # 格式化日期
            '收盤價': df['收盤價'].tolist(), # 收盤價列表
            '每日報酬': df['收盤價'].pct_change().tolist() # 報酬率 = 價格變化百分比
        }
        return data


    def stock_fundamental(self, stock_id):
        """
        取得股票基本面資料（價值投資分析基礎）

        Args:
            stock_id (str): 股票代號

        Returns:
            dict: 包含季度日期、營收成長率、EPS、EPS季增率

        基本面指標說明：
        - 營收成長率：季度營收相比上一季的成長百分比
        - EPS (每股盈餘)：公司獲利能力的核心指標
        - EPS季增率：EPS相比上一季的成長百分比

        注意事項：
        - 某些股票可能無基本面資料（新上市或資料不全）
        - 使用try-except確保程式不會因單一股票錯誤而中斷
        """

        stock_id += '.TW'
        stock = yf.Ticker(stock_id)

        try:
            # 計算季度營收成長率
            # pct_change(-1)：與前一期（-1）比較的百分比變化
            # fill_method=None：不填補缺失值
            quarterly_revenue_growth = np.round(
                stock.quarterly_financials.loc['Total Revenue'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得季度EPS
            quarterly_eps = np.round(
                stock.quarterly_financials.loc['Basic EPS'].dropna().tolist(),
                2
            )
            # 計算EPS季增率
            quarterly_eps_growth = np.round(
                stock.quarterly_financials.loc['Basic EPS'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得日期並格式化
            dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]
            # 組合資料（確保長度一致）
            data = {
                '季日期': dates[:len(quarterly_revenue_growth)],
                '營收成長率': quarterly_revenue_growth.tolist(),
                'EPS': quarterly_eps.tolist(),
                'EPS 季增率': quarterly_eps_growth.tolist()
            }
        except Exception as e:
            # 如果取得資料失敗，回傳空資料
            print(f"無法取得 {stock_id} 基本面資料: {e}")
            data = {'季日期': [], '營收成長率': [], 'EPS': [], 'EPS 季增率': []}

        return data

    def _get_reply(self, content_msg):
        """
        呼叫Gemini API並處理各種回應狀況（內部方法）

        Args:
            content_msg (str): 要傳送給Gemini的提示詞（prompt）

        Returns:
            str: Gemini的回應文字或錯誤訊息

        錯誤處理機制：
        1. finish_reason = 1 (STOP)：正常完成
        2. finish_reason = 2 (MAX_TOKENS)：超過長度限制
        3. finish_reason = 3 (SAFETY)：觸發安全政策
        4. finish_reason = 4 (RECITATION)：引用問題
        5. HTTP 429：API速率限制（免費版：每分鐘5次，每天25次）
        6. HTTP 503：服務暫時不可用
        """

        try:
            # 呼叫Gemini API生成內容
            response = self.model.generate_content(
                content_msg,
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=10000,  # 最大輸出token數（避免過長）
                    temperature=1.0,  # 溫度參數（1.0 = 較有創意，0.0 = 較保守）
                )
            )

            # 檢查是否有回應候選
            if not response.candidates:
                return "模型未返回任何回應"

            candidate = response.candidates[0]
            finish_reason = candidate.finish_reason

            # finish_reason是整數值，代表不同的完成狀態：
            # 0 = UNSPECIFIED (未指定)
            # 1 = STOP (正常完成)
            # 2 = MAX_TOKENS (達到token限制)
            # 3 = SAFETY (安全過濾)
            # 4 = RECITATION (引用問題)
            # 5 = OTHER (其他原因)

            # 根據不同的完成原因處理
            if finish_reason == 1:  # STOP - 正常完成
                # 正常完成
                return candidate.content.parts[0].text

            elif finish_reason == 2:  # MAX_TOKENS - 達到長度限制
                try:
                    partial_text = candidate.content.parts[0].text if candidate.content.parts else ""
                    return f"{partial_text}\n\n [回應被截斷，已達到長度限制]"
                except:
                    return "回應超過長度限制。建議：1)增加 max_output_tokens 2)簡化 prompt"

            elif finish_reason == 3:  # SAFETY - 安全政策
                return "模型因安全政策無法提供回應"

            elif finish_reason == 4:  # RECITATION - 引用問題
                return "回應因引用問題被過濾"

            else:
                return f"未知完成原因: {finish_reason}"

        except Exception as e:
            # 處理API錯誤
            error_msg = str(e)
            # 速率限制錯誤（最常見）
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                return "達到 API 速率限制（免費層：5次/分鐘，25次/天）"
            # 服務暫時不可用
            elif "503" in error_msg:
                return "服務暫時不可用，請稍後重試"
            # 其他錯誤
            else:
                return f"API 錯誤: {error_msg}"


    def stock_gimini_analysis(self, stock_id):
        """
        使用Gemini AI分析單支股票（核心功能）

        Args:
            stock_id (str): 股票代號

        Returns:
            str: Gemini生成的股票分析報告

        分析流程：
        1. 取得股票名稱
        2. 收集技術面資料（價格、報酬率）
        3. 收集基本面資料（營收、EPS）
        4. 將資料摘要後傳送給Gemini
        5. 讓Gemini扮演金融分析師角色，產生專業分析

        Prompt設計要點：
        - 角色設定：明確告訴模型扮演「金融分析師」
        - 數據提供：精簡但完整的關鍵資訊
        - 輸出要求：技術面+基本面+風險評估
        - 語言指定：繁體中文
        """
        # 取得股票名稱
        stock_name = self.stock_info.get_stock_name(stock_id, self.name_df)

        # 收集股價資料
        price_data = self.stock_price(stock_id)
        print('資料收集完畢')
        # 收集基本面資料
        stock_value_data = self.stock_fundamental(stock_id)

        # --- 資料摘要製作 ---
        # 目的：將大量資料濃縮成關鍵指標，避免超過token限制
        summary_parts = []

        # 股價摘要：最新價格 + 兩週漲跌幅
        if price_data and price_data['收盤價']:
            latest = price_data['收盤價'][-1] # 最新收盤價
            first = price_data['收盤價'][0] # 首日收盤價
            change = ((latest - first) / first) * 100 # 計算漲跌百分比
            summary_parts.append(f"股價：{latest:.2f} (兩週 {change:+.2f}%)")

        # 基本面摘要：最新一季的營收成長和EPS
        if stock_value_data and stock_value_data['季日期']:
            q = stock_value_data['季日期'][0] # 季度
            rev = stock_value_data['營收成長率'][0] # 營收成長率
            eps = stock_value_data['EPS'][0] # EPS
            summary_parts.append(f"基本面({q})：營收成長{rev*100:.1f}%, EPS {eps:.2f}")

        # 組合所有摘要
        combined = "；".join(summary_parts)

        # ============ 作業修改區 START ============
        # --- 建構Prompt ---
        # 這是Prompt Engineering的關鍵：如何引導LLM產生專業分析
        """
        'conservative': '''你是保守型財務顧問...
        1. 優先評估下檔風險
        2. 關注股息穩定性
        3. 避免高波動股票...''',

        'aggressive': '''你是成長型投資顧問...
        1. 關注營收和獲利成長動能
        2. 評估產業趨勢和競爭優勢''',

        'balanced': '''你是專業金融分析師...
        平衡評估機會與風險...'''
        """
        content_msg = f'''你是保守型財務顧問，你的角色特性是: 1.優先評估下檔風險和可能虧損,2.關注股息穩定性和財務健全度,3.避免推薦高波動或投機性股票,4.用詞謹慎：「建議謹慎」、「需要注意風險」。根據以下股票數據生成投資建議{stock_name}。
資料：{combined}

    請提供詳細的技術面和基本面見解，並考慮風險因素。繁體中文回應。'''
        # ============ 作業修改區 END ============
        # 呼叫Gemini API
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_sort(self, message):
        """
        讓Gemini為多檔股票評分並排序

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini生成的評分排序結果

        評分標準：
        - 50分為基準
        - 正面因素加分（例如：營收成長、技術面轉強）
        - 負面因素扣分（例如：虧損、下跌趨勢）
        - 綜合技術面+基本面+市場環境

        應用場景：投資組合篩選、資金配置決策
        """

        content_msg = f'''你是專業股票分析師，根據趨勢報告評分(0-100)。
50分為基準，正面消息加分，負面消息扣分。
排序所有股票。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_choice(self, message):
        """
        讓Gemini從多檔股票中選出最佳投資標的

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini選出的最佳股票及理由

        決策邏輯：
        - 即使所有選項都不理想，也要選出相對最佳的
        - 需要說明選擇理由（風險、報酬、時機等）
        - 模擬真實的投資決策情境

        教學價值：
        - 理解LLM如何進行多維度評估
        - 學習投資決策的思考框架
        """

        content_msg = f'''你是專業證券分析師，從趨勢報告中選出最適合投資的一檔股票。
即使都不理想也要選一檔，說明理由。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply

In [ ]:
# 【重要】請修改為你自己的Gemini API Key
# ============ 作業修改區 START ==========
gemini_api_key = 'Hidden API Key'
# ============ 作業修改區 END ============
# 建立股票分析物件
stock_analysis = StockAnalysis(gemini_api_key)

# 單支股價分析

In [ ]:
# ============ 作業修改區 START ==========
# 改成想分析的股票代號
reply = stock_analysis.stock_gimini_analysis('6931')
# ============ 作業修改區 END ============
print(reply)

[*********************100%***********************]  1 of 1 completed


資料收集完畢
好的，您好。

關於您垂詢的青松健康，我已根據您提供的資料，並秉持我一貫的保守型投資原則，為您整理了以下分析與建議。我的核心理念是優先保護本金，追求長期穩健的回報，而非短期的高風險利潤。

---

### **綜合結論**

綜合來看，我**建議對青松健康抱持謹慎觀察的態度**。雖然公司預期保持盈利，但其極低的營收成長性與目前股價所隱含的極高估值之間存在顯著矛盾。對於以風險控制為首要考量的保守型投資者而言，目前的風險回報比並不具吸引力，市場上可能有更符合我們原則的標的。

---

### **詳細見解分析**

#### **一、 基本面分析 (Fundamental Analysis)**

從保守型投資者的角度，我們最關心的是公司的長期穩定性、盈利能力與合理的價格。

1.  **營收與盈利能力**：
    *   **營收成長 (1.0%)**：預估的營收成長率僅為 1.0%，這是一個非常低的數字，顯示公司可能已進入成熟期或面臨成長瓶頸。對於追求長期資產增值的投資者來說，這樣的成長動能略顯不足。
    *   **每股盈餘 (EPS 0.47)**：公司預期是獲利的，這點是正面的。然而，我們必須將此獲利能力與其股價進行比較。

2.  **估值 (Valuation)**：
    *   根據您提供的數據，我們可以估算其 **預估本益比 (Forward P/E)**：
        *   股價 (Price) / 每股盈餘 (EPS) = 59.90 / 0.47 ≈ **127.4 倍**
    *   這是一個**極高的本益比**。通常，如此高的本益比只會出現在市場預期未來會有爆炸性成長的高科技或新創公司。對於一家營收成長僅 1% 的公司而言，127 倍的本益比顯然過高，這意味著股價已嚴重透支了未來的獲利預期。一旦公司未來的表現不如市場預期，股價將面臨大幅修正的**下檔風險**。

3.  **股息與財務健全度**：
    *   您提供的資料未包含股息資訊。對於保守型投資者，穩定且持續的股息發放是衡量公司現金流是否健康、是否願意與股東分享利潤的重要指標。**需要進一步檢視其歷史股息政策與派發率**。
    *   同樣地，我們也需要深入分析其資產負債表，確認公司的負債水平是否過高、現金流量是否穩定。低負債和充裕的自由現

# 收集多檔股票的趨勢報告

In [ ]:
# 建立股票清單
# ============ 作業修改區 START ============
# 請修改為你想分析的股票清單（建議3-5檔，避免超過API限制）
# 台股股票參考：
# 2330 台積電, 2317 鴻海, 2454 聯發科, 2412 中華電, 2881 富邦金
# 2382 廣達, 2303 聯電, 3711 日月光投控, 2891 中信金, 2886 兆豐金
# 生技類 葡萄王: 1707，美吾髮: 1731, 青松: 6931
#
stock_list = ['1707', '1731', '6931']
# ============ 作業修改區 END ============

# 設定儲存路徑
today_time = dt.date.today().strftime('%Y%m%d')
path = './StockGemini/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

print(f"\n{'='*70}")
print(f"開始分析 {len(stock_list)} 檔股票")
print(f"{'='*70}\n")

# 批次處理每一檔股票
for i, stock in enumerate(stock_list, 1):
    print(f"[{i}/{len(stock_list)}] 處理股票: {stock}")

    file_path = f"{path}trend_{stock}_{today_time}.txt"

    if not os.path.exists(file_path):
        try:
            # 分析股票
            print(f"   正在分析...")
            analysis = stock_analysis.stock_gimini_analysis(stock_id=stock)

            # 儲存結果
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(analysis)

            content[stock] = analysis
            print(f"   分析完成並已儲存\n")

            # 速率限制控制
            if i < len(stock_list):
                wait_time = 12
                print(f"   等待 {wait_time} 秒以符合 API 速率限制...\n")
                time.sleep(wait_time)

        except Exception as e:
            error_msg = f"錯誤: {str(e)}"
            print(f"   {error_msg}\n")
            content[stock] = error_msg

    else:
        # 讀取已存在的分析
        with open(file_path, "r", encoding="utf-8") as f:
            content[stock] = f.read()
        print(f"   從快取讀取\n")

print(f"{'='*70}")
print("所有股票分析完成！")
print(f"{'='*70}\n")

print("\n" + "="*70)
print("分析結果")
print("="*70 + "\n")

for stock_id, analysis in content.items():
    stock_name = stock_analysis.stock_info.get_stock_name(stock_id, stock_analysis.name_df)

    print("="*70)
    print(f"【{stock_id} - {stock_name}】")
    print("="*70)
    print(analysis)
    print("\n")


開始分析 3 檔股票

[1/3] 處理股票: 1707
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

   等待 12 秒以符合 API 速率限制...

[2/3] 處理股票: 1731
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

   等待 12 秒以符合 API 速率限制...



[*********************100%***********************]  1 of 1 completed

[3/3] 處理股票: 6931
   正在分析...
資料收集完畢


   分析完成並已儲存

所有股票分析完成！


分析結果

【1707 - 葡萄王】
好的，身為您的保守型財務顧問，我將根據您提供的葡萄王（Grape King）股票數據，以風險控管為優先，為您提供一份審慎的投資評估。

我的分析將優先評估潛在的下檔風險，並著重於營運的穩定性。

---

### **保守型投資顧問對葡萄王的綜合評估**

**總體評估：**
目前葡萄王的股價在短期內表現穩定，但其基本面預測數據呈現出一些值得高度關注的警訊。對於以資本保值和穩定收益為目標的保守型投資者而言，**建議採取謹慎觀望的態度**。以下是詳細的分析：

---

### **一、 基本面見解 (Fundamental Analysis)**

基本面是保守型投資的核心。從您提供的數據來看，存在顯著的潛在隱憂。

1.  **營收成長呈現衰退 (-5.0%)：**
    *   **核心警訊：** 這是最需要注意的風險信號。一家公司的營收是其獲利的根本來源，當營收預期出現負成長，通常意味著公司可能面臨市場萎縮、市佔率下滑或產業競爭加劇等挑戰。
    *   **對保守投資者的意義：** 我們尋求的是業務穩定或能持續成長的企業。營收衰退可能影響未來的盈利能力與股息發放的穩定性，這與我們的核心投資原則相悖。

2.  **預估每股盈餘 (EPS) 2.00 元：**
    *   **本益比評估：** 以目前股價 126.50 元計算，其預估本益比 (P/E Ratio) 高達 **63.25 倍** (126.50 / 2.00)。
    *   **估值過高風險：** 對於一家營收預期衰退的公司而言，超過 60 倍的本益比屬於非常高的水平。這意味著市場給予了極高的期望，但公司的基本面數據卻無法支撐如此樂觀的估值。一旦未來財報不如預期，股價將面臨大幅修正的下檔風險。

3.  **需要進一步評估的項目：**
    *   **股息穩定性：** 作為保守型投資者，股息是重要的考量。我們需要深入研究葡萄王過去的股息發放紀錄、盈餘分配率，並評估在營收衰退的情況下，公司是否仍有能力維持穩定的股息。若EPS下降，可能會影響未來的股息政策。
    *   **財務健全度：** 必須檢視其資產負債表，確認公司的負債比率是否過高、現金流量是否充裕。一家財務穩健的公司較有能力度過營運

# 推薦一檔股票

In [ ]:
reply = stock_analysis.stock_gimini_choice(str(content))
print(reply)

好的，身為專業證券分析師，我已詳細審閱您提供的三份趨勢報告。坦白說，這三檔股票根據所附的數據來看，都存在顯著的風險，並不符合理想的投資標的特質。

然而，根據您的指示，即使都不理想也必須選出一檔。我的選擇將基於「**兩害相權取其輕**」的原則，找出在目前已知的負面因素中，相對具有一絲轉圜餘地或風險較低的標的。

### **總結：我的選擇**

在三者之中，我會選擇 **1731 美吾華** 作為最適合的投資標的。

---

### **分析與決策流程**

我的分析流程分為兩步：首先，排除風險最高的選項；其次，在剩餘的選項中，選擇基本面損害程度相對較輕的一檔。

#### **第一步：排除風險最高的標的 — 6931 青松健康**

我首先排除 **6931 青松健康**，主要原因在於其**極端的估值風險**。

*   **致命缺陷：** 預估本益比高達 **127.4 倍**，但營收成長率卻僅有 **1.0%**。
*   **分析師觀點：** 這是一個極度危險的組合。市場給予一家近乎零成長的公司超過百倍的本益比，意味著股價完全脫離基本面，可能是由題材、籌碼或其他非理性因素支撐。這種股票的下檔風險極大，一旦市場情緒反轉或財報稍有不如預期，股價修正將會非常劇烈且迅速。對任何理性的投資人而言，這種估值水平都構成了無法接受的風險。

#### **第二步：比較剩餘兩者 — 1707 葡萄王 vs. 1731 美吾華**

在排除了青松健康後，我們剩下葡萄王和美吾華。兩者共同的問題是「營收衰退」和「高本益比」。我們需要比較誰的問題更嚴重。

| 指標 | **1707 葡萄王** | **1731 美吾華** | **分析師點評** |
| :--- | :--- | :--- | :--- |
| **預估營收成長** | **-5.0%** | **-1.0%** | **美吾華勝**。營收是企業的命脈，-5.0%的衰退幅度遠比-1.0%嚴重，顯示葡萄王可能面臨更嚴峻的市場競爭或內部營運挑戰。-1.0%的衰退相對溫和，更有可能在短期內回到持平或微幅成長的軌道。 |
| **預估本益比** | **63.25 倍** | **約 70 倍** | **葡萄王略優**。兩者本益比都偏高，但葡萄王的估值壓力相對美吾華稍低一些。然而，這個微小的優勢，並不足以抵銷其嚴重的

# 推薦股票的評分排序

In [ ]:
reply = stock_analysis.stock_gimini_sort(str(content))
print(reply)

好的，身為專業股票分析師，我已詳細審閱您提供的三份由保守型財務顧問撰寫的報告。我的評分將基於報告中揭示的基本面、技術面、估值與風險因子，並以50分為中性基準。

### **個股分析與評分**

---

#### **1. 1707 葡萄王 (Grape King)**

*   **正面因素 (+)**:
    *   短期股價穩定，近兩週微幅上漲1.61%，顯示市場情緒相對平穩。

*   **負面因素 (-)**:
    *   **營收衰退 (-5.0%)**: 這是最嚴重的警訊，代表公司核心業務面臨挑戰，動能堪憂。
    *   **估值極高**: 預估本益比超過63倍，對於一家營收衰退的公司而言，估值風險極高，股價缺乏基本面支撐。
    *   **股息不確定性**: 獲利能力下滑可能影響未來股息的穩定性。

*   **分析師總評**:
    報告明確指出基本面與估值之間存在嚴重矛盾。營收的顯著衰退是硬傷，而過高的本益比則大幅增加了下檔風險。短期股價的穩定更像是風險來臨前的寧靜。

**趨勢報告評分: 19 / 100**

---

#### **2. 1731 美吾華 (Maywufa)**

*   **正面因素 (+)**:
    *   具備股息發放的歷史紀錄，對偏好收益的投資者有基本吸引力。

*   **負面因素 (-)**:
    *   **估值極度昂貴**: 預估本益比高達70倍，與其營運現況完全不匹配，是最大的風險所在。
    *   **營收微幅衰退 (-1.0%)**: 顯示公司成長停滯，缺乏新的成長動能。
    *   **轉投資不確定性**: 報告提及生技轉投資，這為保守型投資組合增添了不必要的投機性與波動風險。
    *   **短期股價疲軟**: 近兩週下跌-0.68%，市場動能偏弱。

*   **分析師總評**:
    美吾華面臨與葡萄王類似的「高估值、低成長」困境，且其本益比更高。雖然營收衰退幅度較小，但轉投資事業帶來的不確定性，進一步拉高了其風險屬性。

**趨勢報告評分: 16 / 100**

---

#### **3. 6931 青松健康 (Ching Song)**

*   **正面因素 (+)**:
    *   公司處於獲利狀態，具備基本營運能力。

*   **負面因